In [1]:
from functools import partial
import numpy as np
import pandas as pd
from scipy.stats import skew, kurtosis
from sklearn.preprocessing import OneHotEncoder
from tqdm.contrib.concurrent import process_map
from gower.gower_dist import *

a = ['a'] * 10
b = list('abcdefghij')
c = ['a'] * 1 + ['b'] * 9
d = ['a'] * 5 + ['b'] * 5
e = list('abcdeabcde')
f = ['a'] * 3 + ['b'] * 3 + ['c'] * 3 + ['d'] * 1
g = ['a'] * 1 + ['b'] * 2 + ['c'] * 3 + ['d'] * 4

process_map(get_cat_weight, [a, b, c, d, e, f, g])


  0%|          | 0/7 [00:00<?, ?it/s]

[0,
 0,
 0.16000000000000003,
 0.24691358024691357,
 0.32098765432098725,
 0.3733333333333334,
 0.40329218106995873]

In [2]:
df = pd.DataFrame({"Sex1": ["M", "M", "F", "F", "F", "M", "M", "F", "F", "F"],
                   "Sex2": ["M", "M", "F", "F", "F", "F", "F", "M", "M", "M"],
                   "Age1": [15] * 10,
                   "Age2": [15, 36, 58, 78, 100, 15, 36, 58, 78, 100]})

In [3]:
x = gower_matrix(df[["Sex1", "Age1"]].to_numpy(), df[["Sex2", "Age2"]].to_numpy(), weight=np.ones(2))
df.loc[np.argsort(np.diag(x))]

  0%|          | 0/10 [00:00<?, ?it/s]

,Sex1,Sex2,Age1,Age2
0,M,M,15,15
1,M,M,15,36
2,F,F,15,58
3,F,F,15,78
4,F,F,15,100
5,M,F,15,15
6,M,F,15,36
7,F,M,15,58
8,F,M,15,78
9,F,M,15,100


In [4]:
x = gower_matrix(df[["Sex1", "Age1"]].to_numpy(), df[["Sex2", "Age2"]].to_numpy())
df.loc[np.argsort(np.diag(x))]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

,Sex1,Sex2,Age1,Age2
0,M,M,15,15
1,M,M,15,36
5,M,F,15,15
6,M,F,15,36
2,F,F,15,58
3,F,F,15,78
7,F,M,15,58
8,F,M,15,78
4,F,F,15,100
9,F,M,15,100
